In [1]:
!pip install yfinance

In [2]:
import yfinance as yf
from datetime import datetime
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
import statsmodels as smt



start_date = datetime(2021,1,1)
end_date = datetime(2022,1,31)
Visa = yf.download('V',start_date ,end_date,index=False)

[*********************100%***********************]  1 of 1 completed


In [3]:
Visa.reset_index(inplace=True)
Visa

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,220.250000,220.250000,214.240005,217.759995,216.078995,10318300
1,2021-01-05,216.309998,218.460007,213.639999,214.509995,212.854065,6869700
2,2021-01-06,213.800003,216.169998,212.070007,212.619995,210.978653,7206200
3,2021-01-07,212.199997,215.220001,212.199997,213.809998,212.159470,10967900
4,2021-01-08,214.169998,216.119995,212.750000,215.449997,213.786819,6513000
...,...,...,...,...,...,...,...
266,2022-01-24,202.179993,202.720001,195.649994,201.850006,201.522141,16054000
267,2022-01-25,199.619995,205.380005,196.990005,202.000000,201.671890,12125500
268,2022-01-26,204.279999,210.860001,203.669998,205.869995,205.535599,10817600
269,2022-01-27,209.500000,210.960007,205.850006,206.149994,205.815140,12009800


In [4]:
df = pd.read_csv('F-F_Research_Data_Factors_daily.csv',skiprows=4)
df

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,19260701,0.10,-0.23,-0.28,0.009
1,19260702,0.45,-0.34,-0.03,0.009
2,19260706,0.17,0.29,-0.38,0.009
3,19260707,0.09,-0.59,0.00,0.009
4,19260708,0.21,-0.38,0.18,0.009
...,...,...,...,...,...
25164,20220126,-0.30,-1.08,0.09,0.000
25165,20220127,-0.78,-1.63,0.77,0.000
25166,20220128,2.45,-0.15,-2.07,0.000
25167,20220131,2.33,1.29,-2.86,0.000


In [5]:
df.rename(columns = {'Unnamed: 0':'Date'},inplace=True)

In [6]:
df=df[:-1]

In [7]:
df['Date']=pd.to_datetime(df['Date'])

<ipython-input-7-182e23b44254>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date']=pd.to_datetime(df['Date'])


In [8]:
df

,Date,Mkt-RF,SMB,HML,RF
0,1926-07-01,0.10,-0.23,-0.28,0.009
1,1926-07-02,0.45,-0.34,-0.03,0.009
2,1926-07-06,0.17,0.29,-0.38,0.009
3,1926-07-07,0.09,-0.59,0.00,0.009
4,1926-07-08,0.21,-0.38,0.18,0.009
...,...,...,...,...,...
25163,2022-01-25,-1.43,-0.60,2.79,0.000
25164,2022-01-26,-0.30,-1.08,0.09,0.000
25165,2022-01-27,-0.78,-1.63,0.77,0.000
25166,2022-01-28,2.45,-0.15,-2.07,0.000


In [9]:
Visa.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [10]:
df_final = Visa.merge(df, how='left', on='Date')

In [11]:
df_final

,Date,Open,High,Low,Close,Adj Close,Volume,Mkt-RF,SMB,HML,RF
0,2021-01-04,220.250000,220.250000,214.240005,217.759995,216.078995,10318300,-1.41,0.22,0.57,0.0
1,2021-01-05,216.309998,218.460007,213.639999,214.509995,212.854065,6869700,0.86,1.21,0.50,0.0
2,2021-01-06,213.800003,216.169998,212.070007,212.619995,210.978653,7206200,0.79,2.13,3.96,0.0
3,2021-01-07,212.199997,215.220001,212.199997,213.809998,212.159470,10967900,1.76,0.32,-0.86,0.0
4,2021-01-08,214.169998,216.119995,212.750000,215.449997,213.786819,6513000,0.51,-0.75,-1.38,0.0
...,...,...,...,...,...,...,...,...,...,...,...
266,2022-01-24,202.179993,202.720001,195.649994,201.850006,201.522141,16054000,0.53,1.81,-0.28,0.0
267,2022-01-25,199.619995,205.380005,196.990005,202.000000,201.671890,12125500,-1.43,-0.60,2.79,0.0
268,2022-01-26,204.279999,210.860001,203.669998,205.869995,205.535599,10817600,-0.30,-1.08,0.09,0.0
269,2022-01-27,209.500000,210.960007,205.850006,206.149994,205.815140,12009800,-0.78,-1.63,0.77,0.0


In [12]:
df_final.set_index(df_final['Date'], inplace=True) 

In [13]:
df_final['Daily_Rtn'] = df_final['Adj Close'].resample('D').last().pct_change().dropna()

In [14]:
df_final=df_final[1:]
df_final

,Date,Open,High,Low,Close,Adj Close,Volume,Mkt-RF,SMB,HML,RF,Daily_Rtn
Date,,,,,,,,,,,,
2021-01-05,2021-01-05,216.309998,218.460007,213.639999,214.509995,212.854065,6869700,0.86,1.21,0.50,0.0,-0.014925
2021-01-06,2021-01-06,213.800003,216.169998,212.070007,212.619995,210.978653,7206200,0.79,2.13,3.96,0.0,-0.008811
2021-01-07,2021-01-07,212.199997,215.220001,212.199997,213.809998,212.159470,10967900,1.76,0.32,-0.86,0.0,0.005597
2021-01-08,2021-01-08,214.169998,216.119995,212.750000,215.449997,213.786819,6513000,0.51,-0.75,-1.38,0.0,0.007670
2021-01-11,2021-01-11,213.960007,214.860001,212.479996,212.889999,211.246567,7353100,-0.51,0.25,1.24,0.0,-0.011882
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,2022-01-24,202.179993,202.720001,195.649994,201.850006,201.522141,16054000,0.53,1.81,-0.28,0.0,-0.019813
2022-01-25,2022-01-25,199.619995,205.380005,196.990005,202.000000,201.671890,12125500,-1.43,-0.60,2.79,0.0,0.000743
2022-01-26,2022-01-26,204.279999,210.860001,203.669998,205.869995,205.535599,10817600,-0.30,-1.08,0.09,0.0,0.019158


In [15]:
X = df_final[['Mkt-RF', 'SMB', 'HML']]/100
y = df_final['Daily_Rtn'] - df_final['RF']
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.372
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     52.57
Date:                Fri, 29 Apr 2022   Prob (F-statistic):           1.03e-26
Time:                        20:40:22   Log-Likelihood:                 788.84
No. Observations:                 270   AIC:                            -1570.
Df Residuals:                     266   BIC:                            -1555.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0008      0.001     -0.966      0.3

In [16]:
X

,const,Mkt-RF,SMB,HML
Date,,,,
2021-01-05,1.0,0.0086,0.0121,0.0050
2021-01-06,1.0,0.0079,0.0213,0.0396
2021-01-07,1.0,0.0176,0.0032,-0.0086
2021-01-08,1.0,0.0051,-0.0075,-0.0138
2021-01-11,1.0,-0.0051,0.0025,0.0124
...,...,...,...,...
2022-01-24,1.0,0.0053,0.0181,-0.0028
2022-01-25,1.0,-0.0143,-0.0060,0.0279
2022-01-26,1.0,-0.0030,-0.0108,0.0009


In [17]:
y

Date
2021-01-05   -0.014925
2021-01-06   -0.008811
2021-01-07    0.005597
2021-01-08    0.007670
2021-01-11   -0.011882
                ...   
2022-01-24   -0.019813
2022-01-25    0.000743
2022-01-26    0.019158
2022-01-27    0.001360
2022-01-28    0.105991
Length: 270, dtype: float64

In [18]:
ff_model.params

const    -0.000783
Mkt-RF    1.230817
SMB      -0.564125
HML       0.166718
dtype: float64